# 数据转换

在数据分析过程中，一般要先行导入数据，并对数据进行探索以获得数据概况信息，然后对数据进行清洗以解决缺省、错误及异常数据。在这之后通常还需要对数据进行转换、加工、处理、衍生已获得更多数据。本节介绍 Pandas 中常用的数据加工处理方法。

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

本节用到用电需求数据集`demand_profile.csv`，包括两个字段：
- `date_time`，日期类型，包括2013年全年
- `energy_kwh`，每个时段的用电量

下面来导入数据：

In [62]:
import os

demand_df = pd.read_csv(os.path.join('..', 'data', 'demand_profile.csv'))
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
date_time     8760 non-null object
energy_kwh    8760 non-null float64
dtypes: float64(1), object(1)
memory usage: 137.0+ KB


## 数据格式转换

除了使用方法 `astype()` 来转换数据类型外，Pandas 还提供了多个函数实现数据转换功能，例如：
- `pd.to_datetime()`：转换数据为日期格式
- `pd.to_timedelta()`：转换数据为时间间隔（timedelta）格式
- `pd.to_numeric()`：转换数据为数值类型

### `pd.to_datetime()`

`pd.to_datetime()`把输入数据转换为 Pandas 日期格式格式，其使用语法为：
```python
pd.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, box=True, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=False)
```
主要参数
- `arg`，输入数据
- `format=None`，数据格式

对于字符串数据，如果不指定格式，Pandas 会使用 `dateutil` 来转换字符串。通过指定格式，性能能大幅提高，甚至百倍之多提升了100倍。

在导入的用电需求数据集中，`date_time`列的数据是字符串。下面使用`pd.to_datetime()`函数转换为日期类型：

In [63]:
df = demand_df.copy()
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%y %H:%M')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
date_time     8760 non-null datetime64[ns]
energy_kwh    8760 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 137.0 KB


### `pd.to_timedelta()`

`pd.to_timedelta()`函数把输入数据转换为 Pandas 的 `timedelta` 格式，其使用语法为：
```python
pd.to_timedelta(arg, unit='ns', box=True, errors='raise')```
主要参数
- `arg`，输入数据
- `unit='ns'`，数据单位，缺省为纳秒。

In [28]:
# 转换数字为 timedelta 格式（单位是秒）
pd.to_timedelta(np.arange(5), unit='s')

TimedeltaIndex(['00:00:00', '00:00:01', '00:00:02', '00:00:03', '00:00:04'], dtype='timedelta64[ns]', freq=None)

In [29]:
# 转换数字为 timedelta 格式（单位是日）
pd.to_timedelta(np.arange(5), unit='d')

TimedeltaIndex(['0 days', '1 days', '2 days', '3 days', '4 days'], dtype='timedelta64[ns]', freq=None)

### `pd.to_numeric()`

`pd.to_numeric()`函数把输入数据转换为 Pandas 的数值格式，其使用语法为：
```python
pd.to_numeric(arg, errors='raise', downcast=None)
```
主要参数
- `arg`，输入数据
- `downcast=None`，类型下降。缺省为None，可以指定`integer`, `signed`, `unsigned`, `float`等。

下面创建一个序列对象，并转换为数值类型：

In [13]:
s = pd.Series(['1.0', '2', -3])
pd.to_numeric(s)

0    1.0
1    2.0
2   -3.0
dtype: float64

缺省情况为转换为浮点数，使用`downcast`参数可以进行类型下降：

In [16]:
# 转换类型为32位浮点数
pd.to_numeric(s, downcast='float')

0    1.0
1    2.0
2   -3.0
dtype: float32

In [17]:
# 转换类型为有符号整数
pd.to_numeric(s, downcast='signed')

0    1
1    2
2   -3
dtype: int8

## 使用映射或函数进行数据转换

对数据进行操作来创建新数据时，常会用如下函数或方法：
- `s.map()`, 对`Series`对象进行元素级别的操作；
- `df.applymap()`，对`DataFrame`对象进行元素级别的操作；
- `df.apply()`，用于`DataFrame`对象行或列上的操作。

### `s.map()`

其使用语法为：
```python
s.map(arg, na_action=None)
```
- `arg`，字典，数组或函数，对元素做映射操作
- `na_action=None`，缺失值处理

下面创建一个序列对象：

In [33]:
ser = pd.Series(range(5))
ser

0    0
1    1
2    2
3    3
4    4
dtype: int64

分别传入字典、函数等进行映射操作：

In [32]:
pd.Series.map?

In [36]:
# 匿名函数
ser.map(lambda x : 'x' + str(x ** 2) )

0     x0
1     x1
2     x4
3     x9
4    x16
dtype: object

In [37]:
# 函数
ser.map(np.sqrt)

0    0.000000
1    1.000000
2    1.414214
3    1.732051
4    2.000000
dtype: float64

In [41]:
# 字典
ser.map({1: 'A', 2: 'B', 3: 'C', 4:'D'})

0    NaN
1      A
2      B
3      C
4      D
dtype: object

### `df.applymap()`

对`DataFrame`对象进行元素级别的操作，其使用方法：
```python
df.applymap(func)
```
主要参数
- `func`，可调用对象。

In [47]:
df = pd.DataFrame({'data1' : np.random.rand(4),
                   'data2' : np.random.rand(4)})
df

,data1,data2
0,0.632560,0.788138
1,0.603291,0.487607
2,0.380650,0.571764
3,0.135461,0.561116


In [53]:
# 匿名函数
df.applymap(lambda x: x ** 3)

,data1,data2
0,0.253108,0.489561
1,0.219574,0.115934
2,0.055154,0.186918
3,0.002486,0.176668


### `df.apply()`

其使用语法为：
```python
df.apply(func, axis=0, broadcast=None, raw=False, reduce=None, result_type=None, args=(), **kwds)
```
主要参数
- `func`，可调用函数，应用于行或列。
- `axis=0`，0表示行，1表示列。
- `broadcast=None`，是否进行广播。
- `raw`，是否直接将数组对象传递给函数。
- `result_type=None`，结果返回类型。

当传入函数为聚合函数，会对行列进行聚合处理；如果为普通函数则与`applymap()`效果类似。

In [61]:
# 累计函数
df.apply(np.sum, axis=1)

0    1.420698
1    1.090898
2    0.952414
3    0.696576
dtype: float64

在用电需求数据集中，在不同时间段上电费费率不同，例如：
- `0--7`时间段，费率为12
- `7--17`时间段，费率为20
- `17--24`时间段，费率为28

使用`df.apply()`来计算费用，首先需要定义函数：

In [65]:
def apply_tariff(kwh, hour):
    """Calculates cost of electricity for given hour."""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

然后应用`df.apply()`来计算使用电费：

In [68]:
df['cost_cents'] = df.apply(
    lambda row: apply_tariff(kwh=row['energy_kwh'], hour=row['date_time'].hour),
    axis=1)

In [69]:
# 结果
df.sample(5)

,date_time,energy_kwh,cost_cents
1502,2013-03-04 14:00:00,0.838,16.760
777,2013-02-02 09:00:00,0.378,7.560
1360,2013-02-26 16:00:00,0.806,16.120
2540,2013-04-16 20:00:00,0.581,16.268
8029,2013-12-01 13:00:00,0.806,16.120


## 